In [1]:
import kagglehub
import numpy as np 
import pandas as pd 
import torch
import os

path_dataset = kagglehub.dataset_download("spellsharp/garbage-data")
print("Path to dataset files:", path_dataset)

Path to dataset files: /kaggle/input/garbage-data


In [2]:
!pip install ultralytics==8.3.29 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 12.2 MB/s eta 0:00:00


In [3]:
import ultralytics, wandb
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6277.7/8062.4 GB disk)


In [4]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wandb_key")

# wandb.login(key =secret_value_0)


In [5]:
from ultralytics import settings
# settings.update({"wandb": True})

In [6]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [7]:
model = YOLO('yolo11s.yaml') 
model = YOLO('yolo11s.pt')  
model = YOLO('yolo11s.yaml').load('yolo11s.pt')


100%|██████████| 18.4M/18.4M [00:00<00:00, 95.3MB/s]


Transferred 499/499 items from pretrained weights


In [8]:
def log_classwise_metrics(trainer):
    """
    This callback is triggered at the end of each validation epoch.
    It extracts per-class metrics and logs them to wandb.
    """
    # Example: assume trainer.val_results contains a list of dictionaries
    # where each dict has keys like 'class_name', 'precision', 'recall', 'mAP50', 'mAP50_95'.
    # (Adapt this to your YOLO11 API.)
    class_metrics = getattr(trainer, 'class_metrics', None)
    if class_metrics is None:
        print("No classwise metrics found; please ensure your validation step computes them.")
        return

    # Option 1: Log as individual scalar metrics:
    log_dict = {}
    for cls in class_metrics:
        name = cls['class_name']
        log_dict[f"{name}/precision"] = cls['precision']
        log_dict[f"{name}/recall"] = cls['recall']
        log_dict[f"{name}/mAP50"] = cls['mAP50']
        log_dict[f"{name}/mAP50-95"] = cls['mAP50_95']
    # Optionally include the current epoch or step:
    wandb.log(log_dict, step=trainer.epoch)

    # Option 2: Log as a wandb Table (for a nice tabular view)
    table = wandb.Table(columns=["class", "precision", "recall", "mAP50", "mAP50-95"])
    for cls in class_metrics:
        table.add_data(cls['class_name'], cls['precision'], cls['recall'],
                       cls['mAP50'], cls['mAP50_95'])
    wandb.log({"Classwise Metrics": table}, step=trainer.epoch)

In [9]:
model.add_callback("on_val_epoch_end", log_classwise_metrics)
model.train(
                            data="/kaggle/input/new-data-yaml/data.yaml",
                            epochs=1000,
                            batch=16,
                            imgsz=640,
                            device="cuda",
                            workers=2,
                            patience=15,
                            multi_scale=False,
                            optimizer="AdamW",
                            lr0=0.0001,
                            lrf=0.1,
                            weight_decay=0.0005,
                            amp=True,
                            project="awm-rihaan-test-new",
                            name="test_run_4",
                            augment=True,
                        )

New https://pypi.org/project/ultralytics/8.3.121 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.yaml, data=/kaggle/input/new-data-yaml/data.yaml, epochs=1000, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=awm-rihaan-test-new, name=test_run_4, exist_ok=False, pretrained=yolo11s.pt, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=Fa

100%|██████████| 755k/755k [00:00<00:00, 17.4MB/s]


Overriding model.yaml nc=80 with nc=44

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 74.9MB/s]


AMP: checks passed ✅


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:32<00:00,  1.95it/s]


                   all       2013       2970      0.591      0.419      0.456      0.348
              Aerosols          3          6          1          0    0.00437    0.00394
          Aluminum can        307        404      0.855      0.921      0.913      0.629
         Aluminum caps          1          1          0          0          0          0
             Cardboard        107        133      0.784      0.827      0.846      0.664
      Combined plastic          2          4          0          0          0          0
Container for household chemicals         34         66      0.519      0.572      0.562      0.346
          Glass bottle         76        100      0.588        0.8      0.693      0.503
         Iron utensils          1          2          0          0          0          0
                Liquid          1          1          0          0          0          0
        Metal shavings          1          1          0          0          0          0
          

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 12.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to awm-rihaan-test-new/test_run_4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  6,  7, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 26, 28, 29, 30, 32, 33, 36, 37, 38, 39, 40, 41, 42, 43])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d442591d420>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,

In [10]:
# import wandb 

In [11]:
# !wandb sync 